In [2]:
!pip install pyecharts
!pip install pyecharts_snapshot

In [1]:
import pyecharts
from tqdm import tqdm
import time
import csv
import pandas as pd
import os
import numpy as np
from pyecharts import options as opts
from pyecharts.charts import Map, Timeline
from pyecharts.globals import ThemeType
import datetime
import random
from pyecharts.charts import Calendar
from pyecharts.charts import Geo
from pyecharts.datasets import register_url
from pyecharts.commons.utils import JsCode
from pyecharts.faker import Collector, Faker
from pyecharts.charts import Bar, Grid, Line, Pie, Tab, Page
import warnings
warnings.filterwarnings("ignore")

In [2]:
df=pd.read_csv('city_temperature.csv')

In [3]:
df.head(5)

,Region,Country,State,City,Month,Day,Year,AvgTemperature
0,Africa,Algeria,NaN,Algiers,1,1,1995,64.2
1,Africa,Algeria,NaN,Algiers,1,2,1995,49.4
2,Africa,Algeria,NaN,Algiers,1,3,1995,48.8
3,Africa,Algeria,NaN,Algiers,1,4,1995,46.4
4,Africa,Algeria,NaN,Algiers,1,5,1995,47.9


In [4]:
df.shape

(2906327, 8)

In [5]:
df.columns

Index(['Region', 'Country', 'State', 'City', 'Month', 'Day', 'Year',
       'AvgTemperature'],
      dtype='object')

In [6]:
df.describe()

,Month,Day,Year,AvgTemperature
count,2.906327e+06,2.906327e+06,2.906327e+06,2.906327e+06
mean,6.469163e+00,1.571682e+01,2.006624e+03,5.600492e+01
std,3.456489e+00,8.800534e+00,2.338226e+01,3.212359e+01
min,1.000000e+00,0.000000e+00,2.000000e+02,-9.900000e+01
25%,3.000000e+00,8.000000e+00,2.001000e+03,4.580000e+01
50%,6.000000e+00,1.600000e+01,2.007000e+03,6.250000e+01
75%,9.000000e+00,2.300000e+01,2.013000e+03,7.550000e+01
max,1.200000e+01,3.100000e+01,2.020000e+03,1.100000e+02


In [7]:
df.isna().sum()

Region                  0
Country                 0
State             1450990
City                    0
Month                   0
Day                     0
Year                    0
AvgTemperature          0
dtype: int64

In [8]:
#delete the missing tenperature rows
df2=df.copy()
df2=df2.drop(df[df['AvgTemperature'].isin([-99])].index)

In [9]:
df2.shape

(2826655, 8)

In [10]:
df2.describe()

,Month,Day,Year,AvgTemperature
count,2.826655e+06,2.826655e+06,2.826655e+06,2.826655e+06
mean,6.464649e+00,1.570960e+01,2.007028e+03,6.037388e+01
std,3.455714e+00,8.800018e+00,7.268831e+00,1.909744e+01
min,1.000000e+00,1.000000e+00,1.995000e+03,-5.000000e+01
25%,3.000000e+00,8.000000e+00,2.001000e+03,4.740000e+01
50%,6.000000e+00,1.600000e+01,2.007000e+03,6.330000e+01
75%,9.000000e+00,2.300000e+01,2.013000e+03,7.590000e+01
max,1.200000e+01,3.100000e+01,2.020000e+03,1.100000e+02


In [11]:
df2['Country'].unique()

array(['Algeria', 'Burundi', 'Benin', 'Central African Republic', 'Congo',
       'Egypt', 'Ethiopia', 'Gabon', 'Gambia', 'Guinea', 'Guinea-Bissau',
       'Ivory Coast', 'Kenya', 'Morocco', 'Madagascar', 'Mauritania',
       'Malawi', 'Mozambique', 'Namibia', 'Nigeria', 'Senegal',
       'Sierra Leone', 'South Africa', 'Togo', 'Tunisia', 'Tanzania',
       'Uganda', 'Zambia', 'Bangladesh', 'China', 'Hong Kong', 'India',
       'Indonesia', 'Japan', 'Kazakhstan', 'Kyrgyzstan', 'Laos',
       'Malaysia', 'Mongolia', 'Myanmar (Burma)', 'Nepal', 'North Korea',
       'Pakistan', 'Philippines', 'Singapore', 'South Korea', 'Sri Lanka',
       'Taiwan', 'Tajikistan', 'Thailand', 'Turkmenistan', 'Uzbekistan',
       'Vietnam', 'Australia', 'New Zealand', 'Albania', 'Austria',
       'Belarus', 'Belgium', 'Bulgaria', 'Croatia', 'Cyprus',
       'Czech Republic', 'Denmark', 'Finland', 'France', 'Germany',
       'Georgia', 'Greece', 'Hungary', 'Iceland', 'Ireland', 'Italy',
       'Latvia', 'Ma

In [12]:
len(df2['City'].unique())

321

In [13]:
df2['Year'].unique()

array([1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005,
       2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016,
       2017, 2018, 2019, 2020], dtype=int64)

In [14]:
#Annual Average Temperature, Maximum Temperature, Minimum Temperature in London, England 1995-2019
london_avg=[0]*25
london_max=[0]*25
london_min=[0]*25
for y in tqdm(range(1995,2020)):    
    london_avg[y-1995]=(round(np.mean(df2[(df2['Year']==y)&(df2['Country']=='United Kingdom')&(df2['City']=='London')]['AvgTemperature'].copy()),2))
    london_max[y-1995]=(round(np.max(df2[(df2['Year']==y)&(df2['Country']=='United Kingdom')&(df2['City']=='London')]['AvgTemperature'].copy()),2))
    london_min[y-1995]=(round(np.min(df2[(df2['Year']==y)&(df2['Country']=='United Kingdom')&(df2['City']=='London')]['AvgTemperature'].copy()),2))

100%|██████████| 25/25 [00:25<00:00,  1.03s/it]


In [15]:
#London 1995-2019 monthly high, average, low temperatures
month_avg=[[0] * 12 for i in range(25)]
month_max=[[0] * 12 for i in range(25)]
month_min=[[0] * 12 for i in range(25)]
for year in tqdm(range(1995,2020)):
    for month in range(1,13):
        month_avg[year-1995][month-1]=(round(np.mean(df2[(df2['Country']=='United Kingdom')&(df2['City']=='London')&(df2['Year']==year)&(df2['Month']==month)]['AvgTemperature']),2))
        month_max[year-1995][month-1]=(round(np.max(df2[(df2['Country']=='United Kingdom')&(df2['City']=='London')&(df2['Year']==year)&(df2['Month']==month)]['AvgTemperature']),2))
        month_min[year-1995][month-1]=(round(np.min(df2[(df2['Country']=='United Kingdom')&(df2['City']=='London')&(df2['Year']==year)&(df2['Month']==month)]['AvgTemperature']),2))

100%|██████████| 25/25 [05:12<00:00, 12.48s/it]


In [17]:
x = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
time=Timeline()
for i in range(1995,2020):
    line=(
        Line(init_opts=opts.InitOpts(width = "1200px", height = "600px", bg_color = '#FFFFFF'))
        .add_xaxis(xaxis_data=x)
        .add_yaxis(
            series_name="Highest",
            y_axis=month_max[i-1995],
            label_opts=opts.LabelOpts(is_show=False),
             itemstyle_opts=opts.ItemStyleOpts(
                 border_width=3,color="#FFCC66"
             ),
            markpoint_opts=opts.MarkPointOpts(
                data=[
                    opts.MarkPointItem(type_="max", name="Max"),
                    opts.MarkPointItem(type_="min", name="Min"),
                ]
            ),
            markline_opts=opts.MarkLineOpts(
                data=[opts.MarkLineItem(type_="average", name="Avg")]
            ),
        )
         .add_yaxis(
             series_name="Lowest",
             y_axis=month_min[i-1995],
             label_opts=opts.LabelOpts(is_show=False),
             itemstyle_opts=opts.ItemStyleOpts(
                 border_width=3,color="#84C1FF"
             ),
             markpoint_opts=opts.MarkPointOpts(
                 data=[
                     opts.MarkPointItem(type_="max", name="Max"),
                     opts.MarkPointItem(type_="min", name="Min"),
                 ]
             ),
             markline_opts=opts.MarkLineOpts(
                 data=[
                     opts.MarkLineItem(type_="average", name="Avg"),
                 ]
             ),
         )
        .set_global_opts(
            title_opts=opts.TitleOpts(title="Monthly Temperatures of London (°F )", subtitle="The monthly highest and lowest temperatures in {}".format(i)),
            tooltip_opts=opts.TooltipOpts(trigger="axis"),
            toolbox_opts=opts.ToolboxOpts(is_show=True),
            xaxis_opts=opts.AxisOpts(type_="category", boundary_gap=False),
            
            yaxis_opts=opts.AxisOpts(max_=100)
        )            
    )          
    time.add(line, "{}".format(i))             
            
time.render("line_chart.html")   
time.render_notebook()


In [18]:
data1=[0]*25
for y in range(1995,2020):
    d3=df[(df['Year']==y)&(df['Country']=='United Kingdom')&(df['City']=='London')].copy()
    s3=[]
    for t in d3['AvgTemperature']:
        if t!=-99:
            s3.append(t)
        else:
            s3.append(np.mean(d3['AvgTemperature']))
    begin = datetime.date(y, 1, 1)
    end = datetime.date(y, 12, 31)
    data1[y-1995] = [
        [str(begin + datetime.timedelta(days=i)), s3[i-1]]
        for i in range((end - begin).days + 1)
    ]

In [19]:

def calendar_1995() -> Calendar:
    i=1995
    ctm = (
        Calendar()
        .add("", data1[i-1995], calendar_opts=opts.CalendarOpts(range_=i))
        .set_global_opts(
            title_opts=opts.TitleOpts(title="Daily temperatures of London in "+"{}".format(i)+" (℉)"),
            visualmap_opts=opts.VisualMapOpts(
                max_=100,
                min_=20,
                orient="horizontal",
                is_piecewise=True,
                pos_top="230px",
                pos_left="100px",
            ),
        )
    )
    return ctm

def calendar_1996() -> Calendar:
    i=1996
    ctm = (
        Calendar()
        .add("", data1[i-1995], calendar_opts=opts.CalendarOpts(range_=i))
        .set_global_opts(
            title_opts=opts.TitleOpts(title="Daily temperatures of London in "+"{}".format(i)+" (℉)"),
            visualmap_opts=opts.VisualMapOpts(
                max_=100,
                min_=20,
                orient="horizontal",
                is_piecewise=True,
                pos_top="230px",
                pos_left="100px",
            ),
        )
    )
    return ctm

def calendar_1997() -> Calendar:
    i=1997
    ctm = (
        Calendar()
        .add("", data1[i-1995], calendar_opts=opts.CalendarOpts(range_=i))
        .set_global_opts(
            title_opts=opts.TitleOpts(title="Daily temperatures of London in "+"{}".format(i)+" (℉)"),
            visualmap_opts=opts.VisualMapOpts(
                max_=100,
                min_=20,
                orient="horizontal",
                is_piecewise=True,
                pos_top="230px",
                pos_left="100px",
            ),
        )
    )
    return ctm

def calendar_1998() -> Calendar:
    i=1998
    ctm = (
        Calendar()
        .add("", data1[i-1995], calendar_opts=opts.CalendarOpts(range_=i))
        .set_global_opts(
            title_opts=opts.TitleOpts(title="Daily temperatures of London in "+"{}".format(i)+" (℉)"),
            visualmap_opts=opts.VisualMapOpts(
                max_=100,
                min_=20,
                orient="horizontal",
                is_piecewise=True,
                pos_top="230px",
                pos_left="100px",
            ),
        )
    )
    return ctm

def calendar_1999() -> Calendar:
    i=1999
    ctm = (
        Calendar()
        .add("", data1[i-1995], calendar_opts=opts.CalendarOpts(range_=i))
        .set_global_opts(
            title_opts=opts.TitleOpts(title="Daily temperatures of London in "+"{}".format(i)+" (℉)"),
            visualmap_opts=opts.VisualMapOpts(
                max_=100,
                min_=20,
                orient="horizontal",
                is_piecewise=True,
                pos_top="230px",
                pos_left="100px",
            ),
        )
    )
    return ctm

def calendar_2000() -> Calendar:
    i=2000
    ctm = (
        Calendar()
        .add("", data1[i-1995], calendar_opts=opts.CalendarOpts(range_=i))
        .set_global_opts(
            title_opts=opts.TitleOpts(title="Daily temperatures of London in "+"{}".format(i)+" (℉)"),
            visualmap_opts=opts.VisualMapOpts(
                max_=100,
                min_=20,
                orient="horizontal",
                is_piecewise=True,
                pos_top="230px",
                pos_left="100px",
            ),
        )
    )
    return ctm

def calendar_2001() -> Calendar:
    i=2001
    ctm = (
        Calendar()
        .add("", data1[i-1995], calendar_opts=opts.CalendarOpts(range_=i))
        .set_global_opts(
            title_opts=opts.TitleOpts(title="Daily temperatures of London in "+"{}".format(i)+" (℉)"),
            visualmap_opts=opts.VisualMapOpts(
                max_=100,
                min_=20,
                orient="horizontal",
                is_piecewise=True,
                pos_top="230px",
                pos_left="100px",
            ),
        )
    )
    return ctm

def calendar_2002() -> Calendar:
    i=2002
    ctm = (
        Calendar()
        .add("", data1[i-1995], calendar_opts=opts.CalendarOpts(range_=i))
        .set_global_opts(
            title_opts=opts.TitleOpts(title="Daily temperatures of London in "+"{}".format(i)+" (℉)"),
            visualmap_opts=opts.VisualMapOpts(
                max_=100,
                min_=20,
                orient="horizontal",
                is_piecewise=True,
                pos_top="230px",
                pos_left="100px",
            ),
        )
    )
    return ctm

def calendar_2003() -> Calendar:
    i=2003
    ctm = (
        Calendar()
        .add("", data1[i-1995], calendar_opts=opts.CalendarOpts(range_=i))
        .set_global_opts(
            title_opts=opts.TitleOpts(title="Daily temperatures of London in "+"{}".format(i)+" (℉)"),
            visualmap_opts=opts.VisualMapOpts(
                max_=100,
                min_=20,
                orient="horizontal",
                is_piecewise=True,
                pos_top="230px",
                pos_left="100px",
            ),
        )
    )
    return ctm

def calendar_2004() -> Calendar:
    i=2004
    ctm = (
        Calendar()
        .add("", data1[i-1995], calendar_opts=opts.CalendarOpts(range_=i))
        .set_global_opts(
            title_opts=opts.TitleOpts(title="Daily temperatures of London in "+"{}".format(i)+" (℉)"),
            visualmap_opts=opts.VisualMapOpts(
                max_=100,
                min_=20,
                orient="horizontal",
                is_piecewise=True,
                pos_top="230px",
                pos_left="100px",
            ),
        )
    )
    return ctm

def calendar_2005() -> Calendar:
    i=2005
    ctm = (
        Calendar()
        .add("", data1[i-1995], calendar_opts=opts.CalendarOpts(range_=i))
        .set_global_opts(
            title_opts=opts.TitleOpts(title="Daily temperatures of London in "+"{}".format(i)+" (℉)"),
            visualmap_opts=opts.VisualMapOpts(
                max_=100,
                min_=20,
                orient="horizontal",
                is_piecewise=True,
                pos_top="230px",
                pos_left="100px",
            ),
        )
    )
    return ctm

def calendar_2006() -> Calendar:
    i=2006
    ctm = (
        Calendar()
        .add("", data1[i-1995], calendar_opts=opts.CalendarOpts(range_=i))
        .set_global_opts(
            title_opts=opts.TitleOpts(title="Daily temperatures of London in "+"{}".format(i)+" (℉)"),
            visualmap_opts=opts.VisualMapOpts(
                max_=100,
                min_=20,
                orient="horizontal",
                is_piecewise=True,
                pos_top="230px",
                pos_left="100px",
            ),
        )
    )
    return ctm

def calendar_2007() -> Calendar:
    i=2007
    ctm = (
        Calendar()
        .add("", data1[i-1995], calendar_opts=opts.CalendarOpts(range_=i))
        .set_global_opts(
            title_opts=opts.TitleOpts(title="Daily temperatures of London in "+"{}".format(i)+" (℉)"),
            visualmap_opts=opts.VisualMapOpts(
                max_=100,
                min_=20,
                orient="horizontal",
                is_piecewise=True,
                pos_top="230px",
                pos_left="100px",
            ),
        )
    )
    return ctm

def calendar_2008() -> Calendar:
    i=2008
    ctm = (
        Calendar()
        .add("", data1[i-1995], calendar_opts=opts.CalendarOpts(range_=i))
        .set_global_opts(
            title_opts=opts.TitleOpts(title="Daily temperatures of London in "+"{}".format(i)+" (℉)"),
            visualmap_opts=opts.VisualMapOpts(
                max_=100,
                min_=20,
                orient="horizontal",
                is_piecewise=True,
                pos_top="230px",
                pos_left="100px",
            ),
        )
    )
    return ctm

def calendar_2009() -> Calendar:
    i=2009
    ctm = (
        Calendar()
        .add("", data1[i-1995], calendar_opts=opts.CalendarOpts(range_=i))
        .set_global_opts(
            title_opts=opts.TitleOpts(title="Daily temperatures of London in "+"{}".format(i)+" (℉)"),
            visualmap_opts=opts.VisualMapOpts(
                max_=100,
                min_=20,
                orient="horizontal",
                is_piecewise=True,
                pos_top="230px",
                pos_left="100px",
            ),
        )
    )
    return ctm

def calendar_2010() -> Calendar:
    i=2010
    ctm = (
        Calendar()
        .add("", data1[i-1995], calendar_opts=opts.CalendarOpts(range_=i))
        .set_global_opts(
            title_opts=opts.TitleOpts(title="Daily temperatures of London in "+"{}".format(i)+" (℉)"),
            visualmap_opts=opts.VisualMapOpts(
                max_=100,
                min_=20,
                orient="horizontal",
                is_piecewise=True,
                pos_top="230px",
                pos_left="100px",
            ),
        )
    )
    return ctm

def calendar_2011() -> Calendar:
    i=2011
    ctm = (
        Calendar()
        .add("", data1[i-1995], calendar_opts=opts.CalendarOpts(range_=i))
        .set_global_opts(
            title_opts=opts.TitleOpts(title="Daily temperatures of London in "+"{}".format(i)+" (℉)"),
            visualmap_opts=opts.VisualMapOpts(
                max_=100,
                min_=20,
                orient="horizontal",
                is_piecewise=True,
                pos_top="230px",
                pos_left="100px",
            ),
        )
    )
    return ctm

def calendar_2012() -> Calendar:
    i=2012
    ctm = (
        Calendar()
        .add("", data1[i-1995], calendar_opts=opts.CalendarOpts(range_=i))
        .set_global_opts(
            title_opts=opts.TitleOpts(title="Daily temperatures of London in "+"{}".format(i)+" (℉)"),
            visualmap_opts=opts.VisualMapOpts(
                max_=100,
                min_=20,
                orient="horizontal",
                is_piecewise=True,
                pos_top="230px",
                pos_left="100px",
            ),
        )
    )
    return ctm

def calendar_2013() -> Calendar:
    i=2013
    ctm = (
        Calendar()
        .add("", data1[i-1995], calendar_opts=opts.CalendarOpts(range_=i))
        .set_global_opts(
            title_opts=opts.TitleOpts(title="Daily temperatures of London in "+"{}".format(i)+" (℉)"),
            visualmap_opts=opts.VisualMapOpts(
                max_=100,
                min_=20,
                orient="horizontal",
                is_piecewise=True,
                pos_top="230px",
                pos_left="100px",
            ),
        )
    )
    return ctm

def calendar_2014() -> Calendar:
    i=2014
    ctm = (
        Calendar()
        .add("", data1[i-1995], calendar_opts=opts.CalendarOpts(range_=i))
        .set_global_opts(
            title_opts=opts.TitleOpts(title="Daily temperatures of London in "+"{}".format(i)+" (℉)"),
            visualmap_opts=opts.VisualMapOpts(
                max_=100,
                min_=20,
                orient="horizontal",
                is_piecewise=True,
                pos_top="230px",
                pos_left="100px",
            ),
        )
    )
    return ctm

def calendar_2015() -> Calendar:
    i=2015
    ctm = (
        Calendar()
        .add("", data1[i-1995], calendar_opts=opts.CalendarOpts(range_=i))
        .set_global_opts(
            title_opts=opts.TitleOpts(title="Daily temperatures of London in "+"{}".format(i)+" (℉)"),
            visualmap_opts=opts.VisualMapOpts(
                max_=100,
                min_=20,
                orient="horizontal",
                is_piecewise=True,
                pos_top="230px",
                pos_left="100px",
            ),
        )
    )
    return ctm

def calendar_2016() -> Calendar:
    i=2016
    ctm = (
        Calendar()
        .add("", data1[i-1995], calendar_opts=opts.CalendarOpts(range_=i))
        .set_global_opts(
            title_opts=opts.TitleOpts(title="Daily temperatures of London in "+"{}".format(i)+" (℉)"),
            visualmap_opts=opts.VisualMapOpts(
                max_=100,
                min_=20,
                orient="horizontal",
                is_piecewise=True,
                pos_top="230px",
                pos_left="100px",
            ),
        )
    )
    return ctm

def calendar_2017() -> Calendar:
    i=2017
    ctm = (
        Calendar()
        .add("", data1[i-1995], calendar_opts=opts.CalendarOpts(range_=i))
        .set_global_opts(
            title_opts=opts.TitleOpts(title="Daily temperatures of London in "+"{}".format(i)+" (℉)"),
            visualmap_opts=opts.VisualMapOpts(
                max_=100,
                min_=20,
                orient="horizontal",
                is_piecewise=True,
                pos_top="230px",
                pos_left="100px",
            ),
        )
    )
    return ctm

def calendar_2018() -> Calendar:
    i=2018
    ctm = (
        Calendar()
        .add("", data1[i-1995], calendar_opts=opts.CalendarOpts(range_=i))
        .set_global_opts(
            title_opts=opts.TitleOpts(title="Daily temperatures of London in "+"{}".format(i)+" (℉)"),
            visualmap_opts=opts.VisualMapOpts(
                max_=100,
                min_=20,
                orient="horizontal",
                is_piecewise=True,
                pos_top="230px",
                pos_left="100px",
            ),
        )
    )
    return ctm

def calendar_2019() -> Calendar:
    i=2019
    ctm = (
        Calendar()
        .add("", data1[i-1995], calendar_opts=opts.CalendarOpts(range_=i))
        .set_global_opts(
            title_opts=opts.TitleOpts(title="Daily temperatures of London in "+"{}".format(i)+" (℉)"),
            visualmap_opts=opts.VisualMapOpts(
                max_=100,
                min_=20,
                orient="horizontal",
                is_piecewise=True,
                pos_top="230px",
                pos_left="100px",
            ),
        )
    )
    return ctm

tab = Tab()
tab.add(calendar_1995(), "1995")
tab.add(calendar_1996(), "1996")
tab.add(calendar_1997(), "1997")
tab.add(calendar_1998(), "1998")
tab.add(calendar_1999(), "1999")
tab.add(calendar_2000(), "2000")
tab.add(calendar_2001(), "2001")
tab.add(calendar_2002(), "2002")
tab.add(calendar_2003(), "2003")
tab.add(calendar_2004(), "2004")
tab.add(calendar_2005(), "2005")
tab.add(calendar_2006(), "2006")
tab.add(calendar_2007(), "2007")
tab.add(calendar_2008(), "2008")
tab.add(calendar_2009(), "2009")
tab.add(calendar_2010(), "2010")
tab.add(calendar_2011(), "2011")
tab.add(calendar_2012(), "2012")
tab.add(calendar_2013(), "2013")
tab.add(calendar_2014(), "2014")
tab.add(calendar_2015(), "2015")
tab.add(calendar_2016(), "2016")
tab.add(calendar_2017(), "2017")
tab.add(calendar_2018(), "2018")
tab.add(calendar_2019(), "2019")
tab.render("carlendar.html")
tab.render_notebook()